In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim,  num_heads,ff_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = ff_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim,
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(ff_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.add = layers.Add()  # instead of `+` to preserve mask
        self.supports_masking = True

    def call(self, inputs, encoder_outputs):
        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, use_causal_mask=True
        )
        out_1 = self.layernorm_1(self.add([inputs, attention_output_1]))

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
        )
        out_2 = self.layernorm_2(self.add([out_1, attention_output_2]))

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(self.add([out_2, proj_output]))

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config
maxlen=30
embed_dim = 128  # Embedding size for each token
num_heads = 8  # Number of attention heads
ff_dim = 128  # Hidden layer size in feed forward network inside transformer
block_num=3 # block_num=6 in the paper
vocab_size=5000
num_bits=500
B=2
def SC_encoder(inputs,flat_flag=True):
    embedding_layer = PositionalEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
   
    for i in range(block_num):
        x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)
    if flat_flag:
        x=layers.Flatten()(x)
    x=layers.Dense(num_bits//B,activation='sigmoid')(x)
    return x
def SC_decoder(decoder_inputs,encoded_seq_inputs,flat_flag=True): 
    y=tf.reshape(encoded_seq_inputs,[-1,num_bits//B])
    y=layers.Dense(embed_dim*maxlen)(y)
    y=layers.LayerNormalization(epsilon=1e-6)(y)
    y=layers.Reshape((maxlen, embed_dim))(y)
    x = PositionalEmbedding(maxlen, vocab_size, embed_dim)(decoder_inputs)
    for i in range(block_num):
        x = TransformerDecoder(embed_dim, num_heads, ff_dim)(x, y)
        x = layers.Dropout(0.5)(x)
    decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
    return decoder_outputs
encoder_inputs = keras.Input(shape=(maxlen), dtype="int64")
encoder_outputs = SC_encoder(encoder_inputs)
encoder0 = keras.Model(encoder_inputs, encoder_outputs)
encoder0.summary()
decoder_inputs = keras.Input(shape=(maxlen), dtype="int64")
encoded_seq_inputs = keras.Input(shape=( num_bits//B))
decoder_outputs = SC_decoder(decoder_inputs, encoded_seq_inputs)
decoder0 = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)
decoder0.summary() 
decoder_outputs = decoder0([decoder_inputs, encoder_outputs])

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 30)]              0         
                                                                 
 positional_embedding_3 (Po  (None, 30, 128)           643840    
 sitionalEmbedding)                                              
                                                                 
 transformer_block_6 (Trans  (None, 30, 128)           561024    
 formerBlock)                                                    
                                                                 
 transformer_block_7 (Trans  (None, 30, 128)           561024    
 formerBlock)                                                    
                                                                 
 transformer_block_8 (Trans  (None, 30, 128)           561024    
 formerBlock)                                              

In [5]:
import pickle
from europarl import TokenizerWrap
with open('tokenizer_5000_1.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
encoder_input_data=tokenizer.tokens_padded
print(print(encoder_input_data.shape))
encoder_inputs = keras.Input(shape=(None,), dtype="int64")
encoder_outputs=encoder0(encoder_inputs)
decoder_inputs = tf.ones_like(encoder_inputs, dtype="int64")
decoder_outputs=decoder0([decoder_inputs,encoder_outputs])
SC_en_de = keras.Model(
    encoder_inputs, decoder_outputs, name="SC_en_de"
)
SC_en_de.summary()

(667875, 30)
None
Model: "SC_en_de"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 tf.ones_like_1 (TFOpLambda  (None, None)                 0         ['input_9[0][0]']             
 )                                                                                                
                                                                                                  
 model_3 (Functional)        (None, 250)                  3287162   ['input_9[0][0]']             
                                                                                                  
 model_4 (Functional)        (None, 30, 5000)             5526664   ['tf.

In [14]:
import os
# X = tokenizer.tokens_padded[0:500000]
# Y = tokenizer.tokens_padded[500000:]
filepath ='SC/'
if not os.path.exists(filepath):
                os.mkdir(filepath[:-1])
if not os.path.exists(filepath+'/his'):
                os.mkdir(filepath)
# class logging(keras.callbacks.Callback):
#     def __init__(self, model, path='SC.txt'):
#         super(logging, self).__init__()

#         self.epochs_since_last_save = 0
#         self.model = model
#         self.path = path

#     def on_epoch_end(self, epoch, logs={}):
#         if (epoch == 0):
#             with open('SC/his/' + self.path, 'w') as f:
#                 # f.write(str(np.shape(HH))+'\n')
#                 f.write('Begin~' + '\n')
#         with open('SC/his/' + self.path, 'a') as f:
#             # f.write(str(np.shape(HH))+'\n')
#             f.write(str(logs.get('loss'))+ '\t'+str(logs.get('val_accuracy')) + '\n')
#        # print('SNR:',random_SNR(10))
#         if (epoch%10==0):
#             encoder0.save_weights(filepath+'SC_en_'+str(epoch)+'.h5')
#             decoder0.save_weights(filepath+'SC_de_'+str(epoch)+'.h5')
# epochs = 100  # SC_en+SC_de step1
# SC_en_de.compile(
#     optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"]
# )
# log=logging(SC_en_de)
# SC_en_de.fit(X,X,batch_size=128, epochs=epochs, validation_data=(Y,Y),callbacks=[log])

In [18]:
# encoder0.save_weights(filepath+'SC_en0.h5')
# decoder0.save_weights(filepath+'SC_de0.h5')  
encoder0.load_weights(filepath+'SC_en0.h5')
decoder0.load_weights(filepath+'SC_de0.h5') 
encoder0.trainable=False
decoder0.trainable=False

In [17]:
num_bits1=400
encoder_inputs = keras.Input(shape=(maxlen), dtype="int64")
encoder_outputs = SC_encoder(encoder_inputs)
encoder1 = keras.Model(encoder_inputs, encoder_outputs)
encoder1.summary()
decoder_inputs = keras.Input(shape=(maxlen), dtype="int64")
encoded_seq_inputs = keras.Input(shape=( num_bits//B))
decoder_outputs = SC_decoder(decoder_inputs, encoded_seq_inputs)
decoder1 = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)
decoder1.summary() 
decoder_outputs = decoder0([decoder_inputs, encoder_outputs])

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, 30)]              0         
                                                                 
 positional_embedding_13 (P  (None, 30, 128)           643840    
 ositionalEmbedding)                                             
                                                                 
 transformer_block_24 (Tran  (None, 30, 128)           561024    
 sformerBlock)                                                   
                                                                 
 transformer_block_25 (Tran  (None, 30, 128)           561024    
 sformerBlock)                                                   
                                                                 
 transformer_block_26 (Tran  (None, 30, 128)           561024    
 sformerBlock)                                            

In [19]:
encoder1.load_weights(filepath+'SC_en0.h5')
decoder1.load_weights(filepath+'SC_de0.h5') 
encoder1.trainable=False
decoder1.trainable=False

In [32]:
import numpy as np
B=2
N=16
d_model=128

def Num2Bit(Num, B):
    Num_ = Num.numpy()
    bit = (np.unpackbits(np.array(Num_, np.uint8), axis=1).reshape(-1, Num_.shape[1], 8)[:, :, (8-B):]).reshape(-1,
                                                                                                            Num_.shape[
                                                                                                                1] * B)
    bit.astype(np.float32)
    return tf.convert_to_tensor(bit, dtype=tf.float32)
# Bit to Number Function Defining
def Bit2Num(Bit, B):
    Bit_ = Bit.numpy()
    Bit_.astype(np.float32)
    Bit_ = np.reshape(Bit_, [-1, int(Bit_.shape[1] / B), B])
    num = np.zeros(shape=np.shape(Bit_[:, :, 1]))
    for i in range(B):
        num = num + Bit_[:, :, i] * 2 ** (B - 1 - i)
    return tf.cast(num, dtype=tf.float32)
#=======================================================================================================================
#=======================================================================================================================
# Quantization and Dequantization Layers Defining
@tf.custom_gradient
def QuantizationOp(x, B,num_bits):
    step = tf.cast((2 ** B), dtype=tf.float32)
    result = tf.cast((tf.round(x * step - 0.5)), dtype=tf.float32)
    dim = result.shape[1]
    result = tf.py_function(func=Num2Bit, inp=[result, B], Tout=tf.float32)
    result = tf.reshape(result, [-1, num_bits])
    def custom_grad(dy):
        grad = dy
        return (grad, grad,grad)
    return result, custom_grad
class QuantizationLayer(tf.keras.layers.Layer):
    def __init__(self, B,num_bits,**kwargs):
        self.B = B
        self.num_bits = num_bits
        super(QuantizationLayer, self).__init__()
    def call(self, x):
        return QuantizationOp(x, self.B, self.num_bits)
    def get_config(self):
        # Implement get_config to enable serialization. This is optional.
        base_config = super(QuantizationLayer, self).get_config()
        base_config['B'] = self.B
        return base_config
@tf.custom_gradient
def DequantizationOp(x, B,num_bits):
    dim = x.shape[1]
    x = tf.py_function(func=Bit2Num, inp=[x, B], Tout=tf.float32)
    x = tf.reshape(x, (-1, num_bits//B))
    step = tf.cast((2 ** B), dtype=tf.float32)
    result = tf.cast((x + 0.5) / step, dtype=tf.float32)
    def custom_grad(dy):
        grad = dy * 1
        return (grad, grad,grad)
    return result, custom_grad
class DeuantizationLayer(tf.keras.layers.Layer):
    def __init__(self, B,num_bits,**kwargs):
        self.B = B
        self.num_bits=num_bits
        super(DeuantizationLayer, self).__init__()
    def call(self, x):
        return DequantizationOp(x, self.B,self.num_bits)
    def get_config(self):
        base_config = super(DeuantizationLayer, self).get_config()
        base_config['B'] = self.B
        return base_config
@tf.custom_gradient
def BSCOp(bits, rate):
    errorbits = tf.keras.backend.random_binomial(shape=tf.shape(bits), p=rate)
    result=tf.reshape(tf.math.floormod(bits+errorbits,2),tf.shape(bits))
    def custom_grad(dy):
        grad = dy * 1
        return (grad, grad)
    return result, custom_grad
class BSC(tf.keras.layers.Layer):
    def __init__(self, rate, **kwargs): #**kwargs```TypeError: __init__() got an unexpected keyword argument 'name'```
        self.rate = rate
        super(BSC, self).__init__()
    def call(self, bits):
        
        return BSCOp(bits,self.rate)


In [40]:
encoder_inputs = keras.Input(shape=(30,), dtype="int64")
SC_en_codeword0 = encoder0(encoder_inputs)
Q_input0=layers.Dense(num_bits//B,activation='sigmoid')(SC_en_codeword0)
bits0=QuantizationLayer(B,num_bits)(Q_input0)#transmission bits
Qmodel0=keras.Model(SC_en_codeword0,bits0)
Qmodel0.summary()
###############
SC_en_codeword1 = encoder1(encoder_inputs)
Q_input1=layers.Dense(num_bits1//B,activation='sigmoid')(SC_en_codeword1)
bits1=QuantizationLayer(B,num_bits1)(Q_input1)#transmission bits
Qmodel1=keras.Model(SC_en_codeword1,bits1)
Qmodel1.summary()
###############
rx_bits0=BSC(0.05)(bits0)
rx_bits1=BSC(0.05)(bits1)
################concat b0 b1#########
Q_output0=DeuantizationLayer(B,num_bits)(rx_bits0)
Q_output1=DeuantizationLayer(B,num_bits1)(rx_bits1)
print(Q_output0.shape)
print(Q_output1.shape)
Q_output=layers.Concatenate()([Q_output0,Q_output1])
########################decoder0##########
SC_de_codeword0=layers.Dense(num_bits//B,activation='sigmoid')(Q_output0)
deQmodel0=keras.Model(Q_output0,SC_de_codeword0)
deQmodel0.summary()
decoder_inputs = tf.ones_like(encoder_inputs, dtype="int64")
decoder_outputs=decoder0([decoder_inputs,SC_de_codeword0])
Qmodel0.load_weights(filepath+'Q_model_90.h5')
deQmodel0.load_weights(filepath+'deQ_model_90.h5')
Qmodel0.trainable=False
deQmodel0.trainable=False
SC_en_Q_deQ_SC_de0 = keras.Model(
    encoder_inputs, decoder_outputs, name="SC_en_Q_deQ_SC_de0"
)
SC_en_Q_deQ_SC_de0.summary()

#############decoder1########
SC_de_codeword1=layers.Dense(num_bits//B,activation='sigmoid')(Q_output)
deQmodel1=keras.Model(Q_output,SC_de_codeword1)
deQmodel1.summary()
decoder_inputs = tf.ones_like(encoder_inputs, dtype="int64")
decoder_outputs=decoder1([decoder_inputs,SC_de_codeword1])

SC_en_Q_deQ_SC_de1 = keras.Model(
    encoder_inputs, decoder_outputs, name="SC_en_Q_deQ_SC_de1"
)
SC_en_Q_deQ_SC_de1.summary()

Model: "model_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_56 (InputLayer)       [(None, 250)]             0         
                                                                 
 dense_135 (Dense)           (None, 250)               62750     
                                                                 
 quantization_layer_16 (Qua  (None, 500)               0         
 ntizationLayer)                                                 
                                                                 
Total params: 62750 (245.12 KB)
Trainable params: 62750 (245.12 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "model_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_57 (InputLayer)       [(None, 250)]             0         
  

 bsc_16 (BSC)                (None, 500)                  0         ['quantization_layer_16[0][0]'
                                                                    ]                             
                                                                                                  
 bsc_17 (BSC)                (None, 400)                  0         ['quantization_layer_17[0][0]'
                                                                    ]                             
                                                                                                  
 deuantization_layer_16 (De  (None, 250)                  0         ['bsc_16[0][0]']              
 uantizationLayer)                                                                                
                                                                                                  
 deuantization_layer_17 (De  (None, 200)                  0         ['bsc_17[0][0]']              
 uantizati

In [41]:
SC_en_Q_deQ_SC_de0.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
SC_en_Q_deQ_SC_de1.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [42]:
SC_en_Q_deQ_SC_de0.evaluate(Y[:100],Y[:100])

4/4 [==============================] - 3s 22ms/step - loss: 2.0829 - accuracy: 0.7423


[2.0829012393951416, 0.7423333525657654]

In [ ]:
encoder1.trainable=True
decoder1.trainable=True
X = tokenizer.tokens_padded[0:500000]
Y = tokenizer.tokens_padded[500000:]
filepath ='SC/'
if not os.path.exists(filepath):
                os.mkdir(filepath[:-1])
if not os.path.exists(filepath+'/his'):
                os.mkdir(filepath)
class logging(keras.callbacks.Callback):
    def __init__(self, model, path='Q_SC.txt'):
        super(logging, self).__init__()

        self.epochs_since_last_save = 0
        self.model = model
        self.path = path

    def on_epoch_end(self, epoch, logs={}):
        if (epoch == 0):
            with open('SC/his/' + self.path, 'w') as f:
                # f.write(str(np.shape(HH))+'\n')
                f.write('Begin~' + '\n')
        with open('SC/his/' + self.path, 'a') as f:
            # f.write(str(np.shape(HH))+'\n')
            f.write(str(logs.get('loss'))+ '\t'+str(logs.get('val_accuracy')) + '\n')
       # print('SNR:',random_SNR(10))
        if (epoch%10==0):
            encoder1.save_weights(filepath+'F_SC_en1_'+str(epoch)+'.h5')
            decoder1.save_weights(filepath+'F_SC_de1_'+str(epoch)+'.h5')
            Qmodel1.save_weights(filepath+'F_Q_model1_'+str(epoch)+'.h5')
            deQmodel1.save_weights(filepath+'F_de_Q_model1_'+str(epoch)+'.h5')
epochs = 100  # SC_en+SC_de step1
SC_en_Q_deQ_SC_de1.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
log=logging(SC_en_Q_deQ_SC_de1)

SC_en_Q_deQ_SC_de1.fit(X,X,batch_size=128, epochs=epochs, validation_data=(Y,Y),callbacks=[log])

Epoch 1/100
1292/3907 [========>.....................] - ETA: 3:55 - loss: 1.3766 - accuracy: 0.7395

In [58]:
test_input=keras.Input(shape=(30,), dtype="int64")
test_bits0=Qmodel0(encoder0(test_input))
test_bits1=Qmodel1(encoder1(test_input))
test_rx_bits0=BSC(0.05)(test_bits0)
test_rx_bits1=BSC(0.05)(test_bits1)
Q_output0=DeuantizationLayer(B,num_bits)(test_rx_bits0)
Q_output1=DeuantizationLayer(B,num_bits1)(test_rx_bits1)
Q_output=layers.Concatenate()([Q_output0,Q_output1])
decoder_inputs = tf.ones_like(test_input, dtype="int64")
SC_de_codeword0=deQmodel0(Q_output0)
SC_de_codeword1=deQmodel1(Q_output)
test_pred0=decoder0([decoder_inputs,SC_de_codeword0])
test_pred1=decoder1([decoder_inputs,SC_de_codeword1])

In [59]:
transmission_model=keras.Model(test_input,[test_rx_bits0,test_rx_bits1])
transmitted_bits0,transmitted_bits1=transmission_model.predict(Y[:100])

4/4 [==============================] - 1s 11ms/step


In [60]:
print(transmitted_bits0)

[[1. 0. 1. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 1. 0.]
 ...
 [0. 1. 1. ... 1. 1. 1.]
 [0. 0. 1. ... 0. 1. 0.]
 [0. 0. 1. ... 0. 1. 0.]]


In [67]:
recevied_model=keras.Model([test_rx_bits0,test_rx_bits1,decoder_inputs],[test_pred0,test_pred1])
recevied_model.summary()
rx1,rx2=recevied_model.predict([transmitted_bits0,transmitted_bits1,np.ones([100,30])])

Model: "model_53"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_67 (InputLayer)       [(None, 500)]                0         []                            
                                                                                                  
 input_68 (InputLayer)       [(None, 400)]                0         []                            
                                                                                                  
 deuantization_layer_28 (De  (None, 250)                  0         ['input_67[0][0]']            
 uantizationLayer)                                                                                
                                                                                                  
 deuantization_layer_29 (De  (None, 200)                  0         ['input_68[0][0]']     

In [69]:
rx1_result=np.argmax(rx1,-1)
rx2_result=np.argmax(rx2,-1)

In [ ]:
result=[]
for i in range(100): #100transmission
    if tokenizer.tokens_to_string(rx1_result[i])==tokenizer.tokens_to_string(Y[i]): #####should be repplaced by CRC32
        result.append(tokenizer.tokens_to_string(rx1_result[i]))
    else:
        result.append(tokenizer.tokens_to_string(rx2_result[i]))
